# Detection of R Peaks

Based on: http://notebooks.pluxbiosignals.com/notebooks/Categories/Detect/r_peaks_rev.html

The electrocardiographic (ECG) signal is notable for its periodicity, a unique feature among physiological signals. This periodic nature underpins the study of cardiac cycle variability, commonly referred to as heart rate variability (HRV), which represents a crucial aspect of ECG analysis. 

Accurate HRV analysis relies on the precise detection of R peaks within the ECG signal. This Jupyter Notebook focuses on this critical task, utilizing the Pan-Tompkins algorithm, which has been adapted to Python by Raja Selvaraj.

## Preparation - Install required module

In [ ]:
!pip install biosignalsnotebooks

## 1 - Import required packages

In [21]:
# biosignalsnotebooks python package
import biosignalsnotebooks as bsnb

# Scientific packages
from numpy import linspace, diff, zeros_like, arange, array
from scipy.signal import filtfilt, butter

## 2 - Load the acquired ECG data from file

In [4]:
# Load data
data, header = bsnb.load_signal("ecg_4000_Hz", get_header=True)

## 3 - Identify the channel used during acquisition

In [5]:
channel = list(data.keys())[0]
print ("Channel: " + str(channel))

Channel: CH1


## 4 - Store sampling rate and acquired data in variables

In [6]:
# Sampling rate.
sr = header["sampling rate"]

# Signal Samples.
signal = data[channel]

### 4.1 - Generate time axis

Generating a time axis allows a more intuitive interpretation of the final results

In [10]:
time = bsnb.generate_time(signal)

## 5 - Simplify the ECG signal (isolate abrupt transitions)

The algorithm proposed by <a href="http://www.robots.ox.ac.uk/~gari/teaching/cdt/A3/readings/ECG/Pan+Tompkins.pdf">Pan and Tompkins </a> is used for the  detection of
the QRS complexes of ECG signals. These complexes are the most obvious parts of ECG tracing (https://en.wikipedia.org/wiki/QRS_complex)
 ### 5.1 - Step 1 of Pan-Tompkins Algorithm - ECG Filtering (Bandpass between 5 and 15 Hz)

In [8]:
# Step 1 of Pan-Tompkins Algorithm
nyquist_sample_rate = sr / 2.
normalized_cut_offs = [5/nyquist_sample_rate, 15/nyquist_sample_rate]
b_coeff, a_coeff = butter(2, normalized_cut_offs, btype='bandpass')[:2]
filtered_signal = filtfilt(b_coeff, a_coeff, signal, padlen=150)

#This step is also available as a single function in biosignalnotebooks
#filtered_signal = bsnb.detect._ecg_band_pass_filter(signal, sr)

In [11]:
bsnb.plot_ecg_pan_tompkins_steps(time, signal, filtered_signal, sr, titles=["Original Signal", "Post Filtering Signal"])

### 5.2 - Step 2 of Pan-Tompkins Algorithm - ECG Differentiation

In [14]:
# Step 2 of Pan-Tompkins Algorithm
differentiated_signal = diff(filtered_signal) # calculate the first derivative

In [13]:
bsnb.plot_ecg_pan_tompkins_steps(time, filtered_signal, differentiated_signal, sr, titles=["Post Filtering Signal", "Post Differentiation Signal"])

### 5.3 - Step 3 of Pan-Tompkins Algorithm - ECG Rectification

In [17]:
# Step 3 of Pan-Tompkins Algorithm
squared_signal = differentiated_signal * differentiated_signal # This will produce a positive signal for all values of t

In [16]:
bsnb.plot_ecg_pan_tompkins_steps(time, filtered_signal, squared_signal, sr, titles=["Post Differentiation Signal", "Post Rectification Signal"])

### 5.4 - Step 4 of Pan-Tompkins Algorithm - ECG Integration ( *Moving window integration* )

#### 5.4.1 - Number of samples within the moving average window

In [19]:
nbr_sampls_int_wind = int(0.080 * sr)
print( "Number of samples: " + str(nbr_sampls_int_wind) )

Number of samples: 320


#### 5.4.2 - Initialisation of the variable that will contain the integrated signal samples

In [22]:
integrated_signal = zeros_like(squared_signal)

#### 5.4.3 - Determination of a cumulative version of "squared_signal"
In the cumulative version of the signal under analysis, his sample value $i$ will be sum of all values included between entry 0 and entry $i$ of the studied signal (in our case "squared_signals").

In [23]:
cumulative_sum = squared_signal.cumsum()

#### 5.4.4 - Estimation of the area/integral below the curve that defines the "squared_signal"
Implicitly, with the current procedure, "squared_signal" is divided into multiple rectangles with fixed width (equal 1 sample) and <a href="https://en.wikipedia.org/wiki/Riemann_sum">height determined by the sample value under analysis </a>.

In [24]:
integrated_signal[nbr_sampls_int_wind:] = (cumulative_sum[nbr_sampls_int_wind:] -
                                           cumulative_sum[:-nbr_sampls_int_wind]) / nbr_sampls_int_wind
integrated_signal[:nbr_sampls_int_wind] = cumulative_sum[:nbr_sampls_int_wind] / arange(1, nbr_sampls_int_wind + 1)

In [25]:
bsnb.plot_ecg_pan_tompkins_steps(time, squared_signal, integrated_signal, sr, titles=["Post Rectification Signal", "Post Integration Signal"])

## 6 - Application of the ECG simplified version to a sequence of peak identification steps
This task is achieved by using a threshold system, with this threshold being dynamically adjusted along the acquisition, like originally proposed by <a href="http://www.robots.ox.ac.uk/~gari/teaching/cdt/A3/readings/ECG/Pan+Tompkins.pdf">Pan and Tompkins </a>

### 6.1 - Initialisation of the R peak detection algorithm

In [27]:
rr_buffer, signal_peak_1, noise_peak_1, threshold = bsnb.detect._buffer_ini(integrated_signal, sr)

### 6.2 - Detection of possible and probable R peaks
Each sample $i$ of our signal is a **possible** peak if his value is greater than the ones at sample $i-1$ and $i+1$. On the other hand, a **probable** peak is a **possible** peak that meets some criteria defined by Pan and Tompkins at their original <a href="http://www.robots.ox.ac.uk/~gari/teaching/cdt/A3/readings/ECG/Pan+Tompkins.pdf">article </a> and synthesized inside the `_detect_peaks` function of the `biosignalsnotebooks` Python package.

In [28]:
probable_peaks, possible_peaks= bsnb.detect._detects_peaks(integrated_signal, sr)

Entry "0" of the returned result contains the list of probable peaks, while entry "1" refers to the possible_peaks.

### 6.3 - Identification of definitive R peaks
Taking into consideration the list of previously detected **probable** peaks, a set of additional criteria were defined by Pan and Tompkins, in order to exclude peaks from the list of **probable** peaks.

In [29]:
definitive_peaks = bsnb.detect._checkup(probable_peaks, integrated_signal, sr, rr_buffer, signal_peak_1, noise_peak_1, threshold)

# Conversion to integer type.
definitive_peaks = array(list(map(int, definitive_peaks)))

### 6.5 - Correcting step
Due to the multiple pre-processing stages there is a small lag in the determined peak positions, which needs to be corrected.

In [30]:
map_integers = definitive_peaks - 40 * (sr / 1000)
definitive_peaks_reph = array(list(map(int, map_integers)))

## 7 - Evolution of the detected peaks (from possible to definitive R peaks)</p>

In [31]:
bsnb.plot_ecg_pan_tompkins_peaks(time, signal, integrated_signal, sr, possible_peaks, probable_peaks, definitive_peaks)

<i>This procedure can be automatically done by <strong>detect_r_peaks</strong> function in <strong>detect</strong> module of <strong><span class="color2">opensignalstools</span></strong> package</i>

In [32]:
detected_peaks = bsnb.detect_r_peaks(signal, sr, time_units=True, plot_result=True)

As demonstrated, the R-peak detection algorithm contains a great number of stages, obviously there are alternative and simpler methodologies. However, to obtain more precise results, more complexity becomes mandatory!

Now you can understand better the complexity behind the heart beat monitoring systems, but you can also program it.